# 거리뷰 이미지 수집

## 1. 구글 street view API 사용
- 거리뷰 이미지를 url로 바로 받아올 수 있음
- 단점: 상세한 거리뷰 이미지가 없음

In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import time

load_dotenv()  
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

def check_streetview_available(lat, lng):
    """
    왜 메타데이터 API를 먼저 쓰냐:
    이미지 API는 없는 곳에 요청해도 과금될 수 있지만
    메타데이터 API는 무료이고 이미지 존재 여부를 먼저 확인할 수 있음
    """
    url = (
        f"https://maps.googleapis.com/maps/api/streetview/metadata"
        f"?location={lat},{lng}"
        f"&key={GOOGLE_API_KEY}"
    )
    res = requests.get(url)
    data = res.json()

    # status가 OK면 이미지 있음, ZERO_RESULTS면 없음
    return data.get("status") == "OK"

def fetch_streetview(lat, lng, zone_id, save_dir="streetview_images"):
    os.makedirs(save_dir, exist_ok=True)
    headings = [0, 90, 180, 270]
    saved_paths = []

    for heading in headings:
        url = (
            f"https://maps.googleapis.com/maps/api/streetview"
            f"?size=640x480"
            f"&location={lat},{lng}"
            f"&heading={heading}"
            f"&fov=90"
            f"&key={GOOGLE_API_KEY}"
        )
        res = requests.get(url)
        if res.status_code == 200:
            path = f"{save_dir}/{zone_id}_h{heading}.jpg"
            with open(path, 'wb') as f:
                f.write(res.content)
            saved_paths.append(path)
        time.sleep(0.2)

    return saved_paths

# 실행
safezone_df = pd.read_csv('./어린이보호구역_위치정보.csv')

available = []
unavailable = []

for idx, row in safezone_df.iterrows():
    lat, lng = row['위도'], row['경도']

    if check_streetview_available(lat, lng):
        paths = fetch_streetview(lat, lng, zone_id=idx)
        available.append(row['대상시설명'])
        print(f"✅ [{idx}] {row['대상시설명']} → {len(paths)}장 수집")
    else:
        unavailable.append(row['대상시설명'])
        print(f"❌ [{idx}] {row['대상시설명']} → 이미지 없음 스킵")

print(f"\n수집 성공: {len(available)}개 / 스킵: {len(unavailable)}개")
print(f"\n이미지 없는 구역: {unavailable}")

❌ [0] 배성유치원 → 이미지 없음 스킵
❌ [1] 이솔유치원 → 이미지 없음 스킵
❌ [2] 성모 유치원 → 이미지 없음 스킵
❌ [3] 판교샘유치원 → 이미지 없음 스킵
❌ [4] 건영장안유치원 → 이미지 없음 스킵
❌ [5] 뽀뽀뽀유치원 → 이미지 없음 스킵
❌ [6] 해나유치원 → 이미지 없음 스킵
❌ [7] 샛별유치원 → 이미지 없음 스킵
❌ [8] 성마르코유치원 → 이미지 없음 스킵
❌ [9] 숲리라유치원 → 이미지 없음 스킵
❌ [10] 즐거운유치원 → 이미지 없음 스킵
❌ [11] 혜성유치원 → 이미지 없음 스킵
❌ [12] 산성어린이집 → 이미지 없음 스킵
❌ [13] 성남어린이집 → 이미지 없음 스킵
❌ [14] 성현어린이집 → 이미지 없음 스킵
❌ [15] 리플플러스어린이집 → 이미지 없음 스킵
❌ [16] 당촌초등학교 → 이미지 없음 스킵
❌ [17] 수내초등학교 → 이미지 없음 스킵
❌ [18] 산성3어린이집 → 이미지 없음 스킵
❌ [19] 은서유치원 → 이미지 없음 스킵
❌ [20] 예원유치원 → 이미지 없음 스킵
❌ [21] 다솜유치원 → 이미지 없음 스킵
❌ [22] 아름다운유치원 → 이미지 없음 스킵
❌ [23] 세화유치원 → 이미지 없음 스킵
❌ [24] 분당중앙유치원 → 이미지 없음 스킵
❌ [25] 보듬이나눔이어린이집 → 이미지 없음 스킵
❌ [26] 운중하나어린이집 → 이미지 없음 스킵
❌ [27] 푸르니이매어린이집 → 이미지 없음 스킵
❌ [28] 성남혜은학교 → 이미지 없음 스킵
❌ [29] 성은특수학교 → 이미지 없음 스킵
❌ [30] 케이디엘피어학원 → 이미지 없음 스킵
❌ [31] 판교초등학교 → 이미지 없음 스킵
❌ [32] 성남동초등학교 → 이미지 없음 스킵
❌ [33] 태평초등학교 → 이미지 없음 스킵
❌ [34] 성수초등학교 → 이미지 없음 스킵
❌ [35] 금빛초등학교 → 이미지 없음 스킵
❌ [36] 왕남초등학교 → 이미지 없음 스킵
❌ [37] 검단초등학교 → 이미지 없음 스킵
❌ [38] 불곡초등

## 2. 카카오맵 API + Selenium을 통해 직접 수집

#### 2-1) 처음 시도
=> 1m 41.5s

In [2]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import pandas as pd

SAVE_DIR = "./roadview_images/image_2-1-1" # 테스트용으로 10개만 저장
DATA_PATH = './schoolzone_location.csv'

os.makedirs(SAVE_DIR, exist_ok=True)

def get_kakao_roadview(lat, lng, facility_name):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1280,720")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    try:
        url = f"https://map.kakao.com/link/roadview/{lat},{lng}"
        driver.get(url)
        time.sleep(4)

        if "준비 중" in driver.page_source:
            print(f"  ❌ [{facility_name}] 로드뷰 없음")
            return None

        # 파일명에 시설명 사용
        safe_name = facility_name.replace('/', '_').replace(' ', '_')
        path = f"{SAVE_DIR}/{safe_name}_북쪽.png"
        driver.save_screenshot(path)
        print(f"  ✅ [{facility_name}] 저장 완료 → {path}")
        return path

    except Exception as e:
        print(f"  ❌ [{facility_name}] 오류: {e}")
        return None

    finally:
        driver.quit()


safezone_df = pd.read_csv(DATA_PATH)
safezone_df = safezone_df.head(10)  # 테스트용으로 상위 10개만

for idx, row in safezone_df.iterrows():
    print(f"[{idx}] {row['시설물명']} 수집 중...")
    get_kakao_roadview(row['위도'], row['경도'], facility_name=row['시설물명'])

[0] 중원초등학교 수집 중...
  ✅ [중원초등학교] 저장 완료 → ./roadview_images/image_2-1-1/중원초등학교_북쪽.png
[1] 하원초등학교 수집 중...


KeyboardInterrupt: 

#### 2-2) 2-1) 개선
=> 1m 4.4s (약 35s 개선)

In [2]:
"""
카카오맵 로드뷰 수집기 (2-1 기반 개선)

2-1에서 동작 확인된 방식을 유지하면서 개선:
  - 브라우저 1회만 열고 재사용 (속도 개선)
  - 파일 크기 검증 추가 (유효성 강화)
  - CSV 결과 저장
  - 포트/서버 없음 (카카오맵 URL 직접 접속이라 불필요)
  
방향 제어, UI 제거는 이 방식에서는 구조적으로 불가하므로 시도하지 않음.
(카카오맵 로드뷰 URL은 heading 파라미터를 지원하지 않고,
 UI 요소는 카카오맵 페이지 자체에 포함되어 있어 제거 불가)
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import pandas as pd

# ============================================================
# 설정
# ============================================================
SAVE_DIR = "./roadview_images/image_2-2"
DATA_PATH = "./schoolzone_location.csv"
MAX_ROWS = 10          # None이면 전체, 숫자면 테스트용
WAIT_SECONDS = 4       # 로드뷰 로딩 대기 (초)

os.makedirs(SAVE_DIR, exist_ok=True)

# ============================================================
# 1) 브라우저 1회 생성 (2-1은 매번 열었지만 여기서는 재사용)
# ============================================================
def create_driver():
    """
    Headless Chrome을 1회만 생성.
    2-1에서는 함수 호출마다 새로 만들었는데,
    여기서는 밖에서 1번 만들고 계속 쓴다.
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1280,720")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options,
    )
    return driver


# ============================================================
# 2) 로드뷰 캡처 (2-1 방식 유지 + 유효성 검증 추가)
# ============================================================
def capture_roadview(driver, lat, lng, facility_name):
    safe_name = facility_name.replace("/", "_").replace(" ", "_")

    try:
        url = f"https://map.kakao.com/link/roadview/{lat},{lng}"
        driver.get(url)
        time.sleep(3)

        # alert 처리 (로드뷰가 있으면 하드웨어 가속 경고가 뜸)
        had_alert = False
        try:
            driver.switch_to.alert.accept()
            had_alert = True
            time.sleep(3)
        except Exception:
            pass

        # 추가 alert 처리
        try:
            driver.switch_to.alert.accept()
        except Exception:
            pass

        time.sleep(2)

        # ★ 핵심 판별: URL에 panoid가 있으면 로드뷰가 로드된 것
        current_url = driver.current_url
        if "panoid" not in current_url:
            print(f"  ⚠️  [{facility_name}] 로드뷰 없음 (panoid 없음) → 스킵")
            return None

        # 스크린샷 저장
        path = f"{SAVE_DIR}/{safe_name}.png"
        driver.save_screenshot(path)

        file_size = os.path.getsize(path)
        if file_size < 10_000:
            os.remove(path)
            print(f"  ⚠️  [{facility_name}] 빈 이미지 → 삭제")
            return None

        print(f"  ✅ [{facility_name}] 저장 → {path} ({file_size/1024:.0f}KB)")
        return path

    except Exception as e:
        try:
            driver.switch_to.alert.accept()
        except Exception:
            pass
        print(f"  ❌ [{facility_name}] 오류: {e}")
        return None

# ============================================================
# 3) 전체 수집
# ============================================================
def collect_all():
    df = pd.read_csv(DATA_PATH)
    if MAX_ROWS is not None:
        df = df.head(MAX_ROWS)

    total = len(df)
    results = []
    success_count = 0
    skip_count = 0

    print("🚀 브라우저 시작...")
    driver = create_driver()
    print(f"📋 총 {total}개 스쿨존 수집 시작\n")

    try:
        for idx, row in df.iterrows():
            name = row["시설물명"]
            lat = row["위도"]
            lng = row["경도"]
            print(f"[{idx + 1}/{total}] {name} 처리 중...")

            path = capture_roadview(driver, lat, lng, name)

            if path:
                success_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "파일경로": path, "상태": "성공",
                })
            else:
                skip_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "파일경로": "", "상태": "로드뷰없음",
                })
    finally:
        driver.quit()
        print("\n🛑 브라우저 종료")

    # 결과 저장
    result_df = pd.DataFrame(results)
    result_path = "./collection_result.csv"
    result_df.to_csv(result_path, index=False, encoding="utf-8-sig")

    print(f"\n{'='*50}")
    print(f"✅ 수집 완료!")
    print(f"   성공: {success_count}개 / 스킵: {skip_count}개 / 전체: {total}개")
    print(f"   결과: {result_path}")
    print(f"   이미지: {SAVE_DIR}")
    print(f"{'='*50}")
    return result_df


if __name__ == "__main__":
    result = collect_all()

🚀 브라우저 시작...
📋 총 10개 스쿨존 수집 시작

[1/10] 중원초등학교 처리 중...
  ✅ [중원초등학교] 저장 → ./roadview_images/image_2-2/중원초등학교.png (1149KB)
[2/10] 하원초등학교 처리 중...
  ✅ [하원초등학교] 저장 → ./roadview_images/image_2-2/하원초등학교.png (1111KB)
[3/10] 중부초등학교 처리 중...
  ✅ [중부초등학교] 저장 → ./roadview_images/image_2-2/중부초등학교.png (1133KB)
[4/10] 상대원초등학교 처리 중...
  ✅ [상대원초등학교] 저장 → ./roadview_images/image_2-2/상대원초등학교.png (1172KB)
[5/10] 도촌초등학교 처리 중...
  ✅ [도촌초등학교] 저장 → ./roadview_images/image_2-2/도촌초등학교.png (971KB)
[6/10] 구미초등학교 처리 중...
  ✅ [구미초등학교] 저장 → ./roadview_images/image_2-2/구미초등학교.png (1193KB)
[7/10] 내정초등학교 처리 중...
  ✅ [내정초등학교] 저장 → ./roadview_images/image_2-2/내정초등학교.png (1259KB)
[8/10] 돌마초등학교 처리 중...
  ⚠️  [돌마초등학교] 로드뷰 없음 (panoid 없음) → 스킵
[9/10] 미금초등학교 처리 중...
  ✅ [미금초등학교] 저장 → ./roadview_images/image_2-2/미금초등학교.png (1184KB)
[10/10] 백현초등학교 처리 중...
  ✅ [백현초등학교] 저장 → ./roadview_images/image_2-2/백현초등학교.png (1312KB)

🛑 브라우저 종료

✅ 수집 완료!
   성공: 9개 / 스킵: 1개 / 전체: 10개
   결과: ./collection_result.csv
   이미지: ./roadview_images/image

#### 2-3) 개선된 버전 

기존 코드의 단점
* 브라우저 : 매 이미지마다 열고 닫음 (142회)
* 이미지 : 이미지당 ~8초 (브라우저 시작 포함)
* 방향 : 제어없음 (기본 방향만)
* 유효성 검증 : "준비 중" 텍스트만 확인
* 결과 : 관리없음

개선된 코드
* 브라우저 : 1회만 열고 재사용
* 이미지 : 이미지당 ~3초
* 방향 : heading 파라미터로 동서남북 가능
* 유효성 검증 : 텍스트 + 파일 크기 검증
* 결과 : CSV로 성공/실패 기록

개선된 코드 주의사항    
 카카오맵 로드뷰 링크 URL에서는 heading 파라미터가 직접 지원되지 않아서 iframe 내부 조작이 제한적일 수 있음.    
 만약 방향 변경이 안 되면 기본 방향 1장만 수집됨   

결과) image_captured_2     

In [ ]:
"""
카카오맵 로드뷰 Selenium 수집기 (개선 버전)
- 브라우저 1회만 실행 (전체 수집 동안 재사용)
- 방향(heading) 제어 가능 (동서남북 4방향)
- 로드뷰 없는 경우 안정적 판별
- 수집 결과 CSV 저장
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd

# ============================================================
# 설정
# ============================================================
SAVE_DIR = "./roadview_images/image_2-2"
DATA_PATH = "./schoolzone_location.csv"
HEADINGS = [0]           # 방향: 0=북, 90=동, 180=남, 270=서  → 4방향이면 [0, 90, 180, 270]
MAX_ROWS = 10           # None이면 전체, 숫자 넣으면 테스트용
WAIT_SECONDS = 3          # 로드뷰 로딩 대기 시간 (초)

os.makedirs(SAVE_DIR, exist_ok=True)

# ============================================================
# 1) 브라우저 1회만 생성
# ============================================================
def create_driver():
    """Headless Chrome 드라이버를 1회 생성하여 반환."""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1280,720")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--enable-webgl")                    # ★ 추가
    options.add_argument("--use-gl=swiftshader")              # ★ 추가: 소프트웨어 WebGL
    options.add_argument("--disable-software-rasterizer")     # ★ 추가

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options,
    )
    return driver


# ============================================================
# 2) 로드뷰 캡처 (heading 제어 포함)
# ============================================================
def capture_roadview(driver, lat, lng, facility_name, headings=[0]):
    """
    카카오맵 로드뷰를 열고, 지정한 heading(방향)별로 스크린샷을 저장한다.
    
    카카오맵 로드뷰 URL 파라미터:
      - /link/roadview/{lat},{lng}  → 기본 진입
      - JavaScript로 heading 변경 가능
    
    Returns:
        list[str]: 저장된 파일 경로 목록 (빈 리스트면 실패)
    """
    saved_paths = []
    safe_name = facility_name.replace("/", "_").replace(" ", "_")
    heading_labels = {0: "북", 90: "동", 180: "남", 270: "서"}

    try:
        # ── 로드뷰 페이지 접속 ──
        url = f"https://map.kakao.com/link/roadview/{lat},{lng}"
        driver.get(url)
        time.sleep(WAIT_SECONDS)

         # ★ alert 처리 (하드웨어 가속 경고 등)
        time.sleep(1)
        try:
            alert = driver.switch_to.alert
            alert.accept()  # 확인 버튼 클릭
        except Exception:
            pass  # alert 없으면 무시
        
        time.sleep(WAIT_SECONDS)

        # ── 로드뷰 사용 가능 여부 판별 ──
        page_source = driver.page_source
        if any(keyword in page_source for keyword in ["준비 중", "로드뷰를 지원하지", "데이터가 없습니다"]):
            print(f"  ⚠️  [{facility_name}] 로드뷰 없음 → 스킵")
            return saved_paths

        # ── heading별 캡처 ──
        for heading in headings:
            label = heading_labels.get(heading, str(heading))

            # JavaScript로 로드뷰 시점(heading) 변경
            # kakao.maps.Roadview 객체의 setViewpoint 사용
            js_code = f"""
            try {{
                var rv = document.querySelector('iframe');
                if (rv) {{
                    // iframe 내부 접근이 안 되면 URL 파라미터로 우회
                    rv.src = rv.src.split('&pan=')[0] + '&pan={heading}';
                }}
            }} catch(e) {{}}
            """
            # 방향 변경 시도 (iframe 구조에 따라 동작 여부가 다를 수 있음)
            if heading != 0:  # 첫 번째(기본)는 변경 불필요
                try:
                    driver.execute_script(js_code)
                    time.sleep(1.5)  # 시점 변경 후 렌더링 대기
                except Exception:
                    pass  # 변경 실패해도 기본 방향으로 캡처

            # ── 스크린샷 저장 ──
            path = f"{SAVE_DIR}/{safe_name}_{label}.png"
            driver.save_screenshot(path)

            # 이미지 크기로 유효성 검증 (너무 작으면 빈 페이지)
            file_size = os.path.getsize(path)
            if file_size < 10_000:  # 10KB 미만이면 빈 이미지로 판단
                os.remove(path)
                print(f"  ⚠️  [{facility_name}] {label}방향 → 빈 이미지, 삭제")
            else:
                saved_paths.append(path)
                print(f"  ✅ [{facility_name}] {label}방향 저장 → {path} ({file_size/1024:.0f}KB)")

    except Exception as e:
        print(f"  ❌ [{facility_name}] 오류: {e}")

    return saved_paths


# ============================================================
# 3) 전체 수집 실행
# ============================================================
def collect_all():
    df = pd.read_csv(DATA_PATH)
    if MAX_ROWS is not None:
        df = df.head(MAX_ROWS)

    total = len(df)
    results = []
    success_count = 0
    skip_count = 0

    # ★ 브라우저 1회만 생성
    print("🚀 브라우저 시작...")
    driver = create_driver()
    print(f"📋 총 {total}개 스쿨존 수집 시작\n")

    try:
        for idx, row in df.iterrows():
            name = row["시설물명"]
            lat = row["위도"]
            lng = row["경도"]
            print(f"[{idx + 1}/{total}] {name} 처리 중...")

            paths = capture_roadview(driver, lat, lng, name, headings=HEADINGS)

            if paths:
                success_count += 1
                results.append({
                    "시설물명": name,
                    "위도": lat,
                    "경도": lng,
                    "저장이미지수": len(paths),
                    "파일경로": "; ".join(paths),
                    "상태": "성공",
                })
            else:
                skip_count += 1
                results.append({
                    "시설물명": name,
                    "위도": lat,
                    "경도": lng,
                    "저장이미지수": 0,
                    "파일경로": "",
                    "상태": "로드뷰없음",
                })

    finally:
        # ★ 전체 수집 끝나면 브라우저 종료
        driver.quit()
        print("\n🛑 브라우저 종료")

    # ── 결과 저장 ──
    result_df = pd.DataFrame(results)
    result_path = "./collection_result.csv"
    result_df.to_csv(result_path, index=False, encoding="utf-8-sig")

    print(f"\n{'='*50}")
    print(f"✅ 수집 완료!")
    print(f"   성공: {success_count}개 / 스킵: {skip_count}개 / 전체: {total}개")
    print(f"   결과 파일: {result_path}")
    print(f"   이미지 폴더: {SAVE_DIR}")
    print(f"{'='*50}")

    return result_df


# ============================================================
# 실행
# ============================================================
if __name__ == "__main__":
    result = collect_all()


🚀 브라우저 시작...
📋 총 10개 스쿨존 수집 시작

[1/10] 중원초등학교 처리 중...
  ✅ [중원초등학교] 북방향 저장 → ./roadview_images/image_captured_2/중원초등학교_북.png (313KB)
[2/10] 하원초등학교 처리 중...
  ✅ [하원초등학교] 북방향 저장 → ./roadview_images/image_captured_2/하원초등학교_북.png (343KB)
[3/10] 중부초등학교 처리 중...

🛑 브라우저 종료


KeyboardInterrupt: 

#### 2-4) 개선된 버전2

기존 코드 (Selenium + 카카오맵 URL)

매 이미지마다 브라우저 열고 닫음 (142회)
이미지당 ~8초
방향 제어 없음 (기본 방향만)
"준비 중" 텍스트로만 유효성 판별
결과 관리 없음
지도 화면 + 사이드바 + 팝업이 같이 캡처됨
하드웨어 가속 alert 에러 발생

개선 코드2 (Selenium + 카카오 JS SDK 로컬 HTML)

브라우저 1회만 열고 재사용
이미지당 ~3초
viewpoint.pan으로 heading(동서남북) 직접 제어
panoId === null로 정확한 판별 + 파일 크기 검증
CSV로 성공/실패 기록
로드뷰만 풀스크린 캡처 (UI 요소 없음)
SDK 직접 사용이라 alert 없음

결과) image2에 저장

In [3]:
"""
카카오맵 로드뷰 Selenium 수집기 (v2 — JS SDK 직접 사용)

핵심 변경:
  map.kakao.com/link/roadview/ URL 대신,
  카카오 JS SDK를 직접 사용하는 로컬 HTML을 만들어서
  로드뷰만 전체화면으로 띄움 → UI 요소 없이 깔끔한 캡처 가능.

사전 준비:
  1. https://developers.kakao.com 에서 JavaScript 앱키 발급
  2. 아래 KAKAO_JS_KEY에 본인 앱키 입력
  3. pip install selenium webdriver-manager pandas
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  # .env 파일 로드
KAKAO_JS_KEY = os.getenv("KAKAO_JS_KEY")  
SAVE_DIR     = "./roadview_images/2-3"
DATA_PATH    = "./schoolzone_location.csv"
MAX_ROWS     = 10       # None=전체, 숫자=테스트용
WAIT_SECONDS = 5          # 로드뷰 렌더링 대기 (초)
WINDOW_W     = 1280
WINDOW_H     = 720

os.makedirs(SAVE_DIR, exist_ok=True)

# ERROR 해결 위해 로컬 HTTP 서버로 HTML 제공
import threading
import http.server
import socketserver

# ============================================================
# 0) 로컬 HTTP 서버 (백그라운드)
# ============================================================
SERVER_PORT = 8765
_httpd = None  # 전역 변수로 서버 인스턴스 관리

def start_local_server():
    global _httpd
    
    # 이미 서버가 실행 중이면 먼저 종료
    if _httpd is not None:
        try:
            _httpd.shutdown()
            _httpd.server_close()
        except Exception:
            pass

    handler = http.server.SimpleHTTPRequestHandler

    # SO_REUSEADDR: 포트를 즉시 재사용할 수 있게 설정
    socketserver.TCPServer.allow_reuse_address = True
    
    # httpd = socketserver.TCPServer(("", SERVER_PORT), handler)
    _httpd = socketserver.TCPServer(("", SERVER_PORT), handler)
    # thread = threading.Thread(target=httpd.serve_forever, daemon=True)
    thread = threading.Thread(target=_httpd.serve_forever, daemon=True)
    thread.start()
    print(f"🌐 로컬 서버 시작: http://localhost:{SERVER_PORT}")
    # return httpd
    return _httpd


# ============================================================
# 1) 로드뷰 전용 HTML 생성
# ============================================================
def create_roadview_html(lat, lng, heading=0):
    """
    카카오 JS SDK로 로드뷰만 전체화면 렌더링하는 HTML 문자열 반환.
    - 사이드바, 지도, UI 버튼 없이 로드뷰 파노라마만 표시
    - heading으로 카메라 방향 제어 가능
    - 로드뷰 없는 좌표면 document.title을 'NO_ROADVIEW'로 변경
    """
    return f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>LOADING</title>
    <style>
        * {{ margin: 0; padding: 0; }}
        html, body, #roadview {{ width: 100%; height: 100%; overflow: hidden; }}
    </style>
</head>
<body>
    <div id="roadview"></div>
    <script src="//dapi.kakao.com/v2/maps/sdk.js?appkey={KAKAO_JS_KEY}"></script>
    <script>
        var container = document.getElementById('roadview');
        var roadview = new kakao.maps.Roadview(container);
        var client = new kakao.maps.RoadviewClient();
        var position = new kakao.maps.LatLng({lat}, {lng});

        client.getNearestPanoId(position, 100, function(panoId) {{
            if (panoId === null) {{
                document.title = 'NO_ROADVIEW';
            }} else {{
                roadview.setPanoId(panoId, position);
                // 로드뷰 초기화 완료 후 heading(방향) 설정
                kakao.maps.event.addListener(roadview, 'init', function() {{
                    var vp = roadview.getViewpoint();
                    vp.pan = {heading};
                    vp.tilt = 0;
                    roadview.setViewpoint(vp);
                    // 렌더링 완료 신호
                    setTimeout(function() {{
                        document.title = 'ROADVIEW_READY';
                    }}, 1500);
                }});
            }}
        }});

        // 타임아웃 — 10초 내 로드 안 되면 실패 처리
        setTimeout(function() {{
            if (document.title === 'LOADING') {{
                document.title = 'NO_ROADVIEW';
            }}
        }}, 10000);
    </script>
</body>
</html>"""


# ============================================================
# 2) 브라우저 생성
# ============================================================
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument(f"--window-size={WINDOW_W},{WINDOW_H}")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--enable-webgl")
    options.add_argument("--use-gl=swiftshader")
    options.add_argument("--disable-software-rasterizer")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options,
    )
    return driver


# ============================================================
# 3) 로드뷰 캡처
# ============================================================
def capture_roadview(driver, lat, lng, facility_name, heading=0):
    """
    로컬 HTML을 data URL로 로드하여 로드뷰 캡처.
    document.title로 로딩 완료/실패를 판별.
    """
    safe_name = facility_name.replace("/", "_").replace(" ", "_")
    heading_labels = {0: "북", 90: "동", 180: "남", 270: "서"}
    label = heading_labels.get(heading, str(heading))

    # HTML 생성 → 임시 파일로 저장
    html_content = create_roadview_html(lat, lng, heading)
    html_path = os.path.abspath("_roadview_temp.html")
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_content)

    try:
        # # 로컬 HTML 열기
        # driver.get(f"file:///{html_path}")

        # 변경 (localhost → SDK 정상 로딩)
        driver.get(f"http://localhost:{SERVER_PORT}/_roadview_temp.html")

        # 로드뷰 렌더링 완료 대기 (title 변경 감지)
        max_wait = WAIT_SECONDS + 5
        elapsed = 0
        while elapsed < max_wait:
            title = driver.title
            if title == "ROADVIEW_READY":
                break
            elif title == "NO_ROADVIEW":
                print(f"  ⚠️  [{facility_name}] 로드뷰 없음 → 스킵")
                return None
            time.sleep(0.5)
            elapsed += 0.5

        if driver.title != "ROADVIEW_READY":
            print(f"  ⚠️  [{facility_name}] 로딩 타임아웃 → 스킵")
            return None

        # 추가 대기 (렌더링 안정화)
        time.sleep(1)

        # 스크린샷 저장
        path = f"{SAVE_DIR}/{safe_name}_{label}.png"
        driver.save_screenshot(path)

        # 유효성 검증
        file_size = os.path.getsize(path)
        if file_size < 10_000:
            os.remove(path)
            print(f"  ⚠️  [{facility_name}] {label}방향 → 빈 이미지, 삭제")
            return None

        print(f"  ✅ [{facility_name}] {label}방향 저장 → {path} ({file_size/1024:.0f}KB)")
        return path

    except Exception as e:
        # alert 처리
        try:
            alert = driver.switch_to.alert
            alert.accept()
        except Exception:
            pass
        print(f"  ❌ [{facility_name}] 오류: {e}")
        return None


# ============================================================
# 4) 전체 수집
# ============================================================
def collect_all():
    df = pd.read_csv(DATA_PATH)
    if MAX_ROWS is not None:
        df = df.head(MAX_ROWS)

    total = len(df)
    results = []
    success_count = 0
    skip_count = 0
    headings = [0]  # 필요시 [0, 90, 180, 270] 으로 변경

    print("🌐 로컬 서버 시작...")
    httpd = start_local_server()

    print("🚀 브라우저 시작...")
    driver = create_driver()
    print(f"📋 총 {total}개 스쿨존 수집 시작\n")

    try:
        for idx, row in df.iterrows():
            name = row["시설물명"]
            lat  = row["위도"]
            lng  = row["경도"]
            print(f"[{idx + 1}/{total}] {name} 처리 중...")

            saved_paths = []
            for h in headings:
                path = capture_roadview(driver, lat, lng, name, heading=h)
                if path:
                    saved_paths.append(path)

            if saved_paths:
                success_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "저장이미지수": len(saved_paths),
                    "파일경로": "; ".join(saved_paths),
                    "상태": "성공",
                })
            else:
                skip_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "저장이미지수": 0, "파일경로": "", "상태": "로드뷰없음",
                })
    finally:
        driver.quit()
        # 임시 파일 정리
        if os.path.exists("_roadview_temp.html"):
            os.remove("_roadview_temp.html")
        print("\n🛑 브라우저 종료")

    # 결과 저장
    result_df = pd.DataFrame(results)
    result_path = "./collection_result.csv"
    result_df.to_csv(result_path, index=False, encoding="utf-8-sig")

    print(f"\n{'='*50}")
    print(f"✅ 수집 완료!")
    print(f"   성공: {success_count}개 / 스킵: {skip_count}개 / 전체: {total}개")
    print(f"   결과: {result_path}")
    print(f"   이미지: {SAVE_DIR}")
    print(f"{'='*50}")
    return result_df


if __name__ == "__main__":
    result = collect_all()

🌐 로컬 서버 시작...
🌐 로컬 서버 시작: http://localhost:8765
🚀 브라우저 시작...
📋 총 10개 스쿨존 수집 시작

[1/10] 중원초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 18:05:35] "GET /_roadview_temp.html HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2026 18:05:35] code 404, message File not found
127.0.0.1 - - [28/Feb/2026 18:05:35] "GET /favicon.ico HTTP/1.1" 404 -


  ⚠️  [중원초등학교] 로딩 타임아웃 → 스킵
[2/10] 하원초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 18:05:45] "GET /_roadview_temp.html HTTP/1.1" 200 -



🛑 브라우저 종료


KeyboardInterrupt: 

## 3. 카카오맵 API 이용해 이미지 다운로드

In [ ]:
import requests
import pandas as pd
import os
import time
from dotenv import load_dotenv

load_dotenv()  
KAKAO_API_KEY = os.getenv("KAKAO_API_KEY")

SAVE_DIR = "./roadview_images/image_3"
DATA_PATH = './schoolzone_location.csv'

os.makedirs(SAVE_DIR, exist_ok=True)

def get_nearest_pano_id(lat, lng, radius=100):
    """
    입력한 좌표 반경 내에서 가장 가까운 로드뷰 파노라마 ID를 반환.
    radius: 탐색 반경(m), 최대 200m 권장
    """
    url = "https://dapi.kakao.com/v2/local/search/roadview.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "x": lng,       # 경도 (longitude)
        "y": lat,       # 위도 (latitude)
        "radius": radius
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"  ❌ API 오류: {response.status_code} - {response.text}")
        return None

    data = response.json()

    if not data.get("documents"):
        print(f"  ⚠️ 반경 {radius}m 내 로드뷰 없음")
        return None

    # documents는 가까운 순서로 정렬되어 반환됨
    nearest = data["documents"][0]
    pano_id = nearest["id"]
    pano_lat = nearest["y"]
    pano_lng = nearest["x"]
    print(f"  ✅ 파노라마 ID: {pano_id} | 좌표: ({pano_lat}, {pano_lng})")
    return pano_id

def download_roadview_image(pano_id, facility_name, direction=0):
    """
    파노라마 ID로 로드뷰 정적 이미지를 다운로드.
    direction: 카메라 방향 (0=북, 90=동, 180=남, 270=서)
    카카오 로드뷰 정적 이미지 API 엔드포인트 사용
    """
    url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"  # 좌표변환용 아님
    
    # 카카오 로드뷰 스태틱 이미지 URL 형식
    image_url = (
        f"https://dapi.kakao.com/v2/map/roadview/static/image"
        f"?panoid={pano_id}"
        f"&panoType=default"
        f"&width=800&height=600"
        f"&heading={direction}"  # 카메라 방향
        f"&pitch=0"              # 카메라 상하 각도 (0=수평)
        f"&fov=90"               # 시야각
    )

    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    response = requests.get(image_url, headers=headers)

    if response.status_code != 200:
        print(f"  ❌ 이미지 다운로드 실패: {response.status_code}")
        return None

    safe_name = facility_name.replace('/', '_').replace(' ', '_')
    direction_label = {0: "북", 90: "동", 180: "남", 270: "서"}.get(direction, str(direction))
    save_path = f"{SAVE_DIR}/{safe_name}_{direction_label}.png"

    with open(save_path, "wb") as f:
        f.write(response.content)

    print(f"  💾 저장 완료 → {save_path}")
    return save_path

def collect_all(directions=[0, 90, 180, 270], max_rows=None):
    """
    CSV의 모든 스쿨존에 대해 동서남북 4방향 로드뷰 이미지를 수집.
    directions: 저장할 방향 목록 (필요에 따라 줄일 수 있음)
    max_rows: 최대 처리할 행 수 (None이면 전체 처리)
    """
    df = pd.read_csv(DATA_PATH)
    if max_rows is not None:
        df = df.head(max_rows)  # 상위 N개 행만 슬라이싱
        
    results = []

    for idx, row in df.iterrows():
        name = row['시설물명']
        lat = row['위도']
        lng = row['경도']
        print(f"\n[{idx+1}/{len(df)}] {name} 처리 중...")

        pano_id = get_nearest_pano_id(lat, lng)

        if pano_id is None:
            results.append({"시설물명": name, "위도": lat, "경도": lng, "pano_id": None, "상태": "로드뷰없음"})
            continue

        saved_paths = []
        for d in directions:
            path = download_roadview_image(pano_id, name, direction=d)
            if path:
                saved_paths.append(path)
            time.sleep(0.3)  # API 호출 간격 (초당 제한 방지)

        results.append({
            "시설물명": name,
            "위도": lat,
            "경도": lng,
            "pano_id": pano_id,
            "저장이미지수": len(saved_paths),
            "상태": "성공"
        })

    # 수집 결과 저장
    result_df = pd.DataFrame(results)
    result_df.to_csv("./collection_result.csv", index=False, encoding="utf-8-sig")
    print("\n✅ 전체 수집 완료. 결과: collection_result.csv")
    return result_df


# 실행
result = collect_all(directions=[0], max_rows=10)   # 테스트용으로 북쪽 방향만, 최대 10개 구역만 수집
# result = collect_all(directions=[0])  # 일단 북쪽만 테스트하려면 [0]만
# result = collect_all()              # 4방향 전체 수집


[1/10] 중원초등학교 처리 중...
  ❌ API 오류: 404 - {"errorType":"ResourceNotFound","message":"'GET /v2/local/search/roadview.json?x=127.167857&y=37.437855&radius=100' is not matched"}

[2/10] 하원초등학교 처리 중...
  ❌ API 오류: 404 - {"errorType":"ResourceNotFound","message":"'GET /v2/local/search/roadview.json?x=127.170236&y=37.446255&radius=100' is not matched"}

[3/10] 중부초등학교 처리 중...
  ❌ API 오류: 404 - {"errorType":"ResourceNotFound","message":"'GET /v2/local/search/roadview.json?x=127.164951&y=37.451546&radius=100' is not matched"}

[4/10] 상대원초등학교 처리 중...
  ❌ API 오류: 404 - {"errorType":"ResourceNotFound","message":"'GET /v2/local/search/roadview.json?x=127.181591&y=37.442382&radius=100' is not matched"}

[5/10] 도촌초등학교 처리 중...
  ❌ API 오류: 404 - {"errorType":"ResourceNotFound","message":"'GET /v2/local/search/roadview.json?x=127.158024&y=37.41473&radius=100' is not matched"}

[6/10] 구미초등학교 처리 중...
  ❌ API 오류: 404 - {"errorType":"ResourceNotFound","message":"'GET /v2/local/search/roadview.json?x=127.1183

#### 3-2) ERROR : 카카오맵 주소 변경

In [ ]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
KAKAO_API_KEY = os.getenv("KAKAO_API_KEY")  # REST API 키

SAVE_DIR = "./roadview_images/image_3-1"
DATA_PATH = './schoolzone_location.csv'
os.makedirs(SAVE_DIR, exist_ok=True)


def get_nearest_pano_id(lat, lng, radius=100):
    """좌표 주변 가장 가까운 로드뷰 파노라마 ID 조회"""
    url = "https://dapi.kakao.com/v2/local/search/roadview.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"x": lng, "y": lat, "radius": radius}

    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code != 200:
        print(f"  ❌ API 오류: {resp.status_code}")
        return None

    docs = resp.json().get("documents", [])
    if not docs:
        print(f"  ⚠️ 반경 {radius}m 내 로드뷰 없음")
        return None

    pano = docs[0]
    print(f"  ✅ 파노라마 ID: {pano['id']}")
    return pano["id"]


def download_roadview_image(pano_id, facility_name, heading=0):
    """파노라마 ID로 정적 로드뷰 이미지 다운로드"""
    # 카카오 로드뷰 정적 이미지 URL
    image_url = (
        f"https://map.kakao.com/roadviewimage"
        f"?panoid={pano_id}"
        f"&width=800&height=600"
        f"&heading={heading}&pitch=0&fov=90"
    )

    resp = requests.get(image_url)
    if resp.status_code != 200 or len(resp.content) < 5000:
        print(f"  ❌ 이미지 다운로드 실패: {resp.status_code}")
        return None

    safe_name = facility_name.replace('/', '_').replace(' ', '_')
    label = {0: "북", 90: "동", 180: "남", 270: "서"}.get(heading, str(heading))
    save_path = f"{SAVE_DIR}/{safe_name}_{label}.png"

    with open(save_path, "wb") as f:
        f.write(resp.content)

    print(f"  💾 저장 → {save_path} ({len(resp.content)/1024:.0f}KB)")
    return save_path


def collect_all(directions=[0], max_rows=10):
    df = pd.read_csv(DATA_PATH)
    if max_rows:
        df = df.head(max_rows)

    results = []
    success = 0

    for idx, row in df.iterrows():
        name, lat, lng = row['시설물명'], row['위도'], row['경도']
        print(f"\n[{idx+1}/{len(df)}] {name}")

        pano_id = get_nearest_pano_id(lat, lng)
        if not pano_id:
            results.append({"시설물명": name, "상태": "로드뷰없음"})
            continue

        paths = []
        for h in directions:
            p = download_roadview_image(pano_id, name, heading=h)
            if p:
                paths.append(p)

        if paths:
            success += 1
            results.append({"시설물명": name, "상태": "성공", "파일": "; ".join(paths)})
        else:
            results.append({"시설물명": name, "상태": "이미지실패"})

    print(f"\n✅ 완료: 성공 {success}/{len(df)}")
    pd.DataFrame(results).to_csv("./collection_result.csv", index=False, encoding="utf-8-sig")
    return results

# 실행
collect_all(directions=[0, 90, 180, 270], max_rows=10)


[1/10] 중원초등학교
  ❌ API 오류: 404

[2/10] 하원초등학교
  ❌ API 오류: 404

[3/10] 중부초등학교
  ❌ API 오류: 404

[4/10] 상대원초등학교
  ❌ API 오류: 404

[5/10] 도촌초등학교
  ❌ API 오류: 404

[6/10] 구미초등학교
  ❌ API 오류: 404

[7/10] 내정초등학교
  ❌ API 오류: 404

[8/10] 돌마초등학교
  ❌ API 오류: 404

[9/10] 미금초등학교
  ❌ API 오류: 404

[10/10] 백현초등학교
  ❌ API 오류: 404

✅ 완료: 성공 0/10


[{'시설물명': '중원초등학교', '상태': '로드뷰없음'},
 {'시설물명': '하원초등학교', '상태': '로드뷰없음'},
 {'시설물명': '중부초등학교', '상태': '로드뷰없음'},
 {'시설물명': '상대원초등학교', '상태': '로드뷰없음'},
 {'시설물명': '도촌초등학교', '상태': '로드뷰없음'},
 {'시설물명': '구미초등학교', '상태': '로드뷰없음'},
 {'시설물명': '내정초등학교', '상태': '로드뷰없음'},
 {'시설물명': '돌마초등학교', '상태': '로드뷰없음'},
 {'시설물명': '미금초등학교', '상태': '로드뷰없음'},
 {'시설물명': '백현초등학교', '상태': '로드뷰없음'}]

In [5]:
"""
카카오맵 로드뷰 Selenium 수집기 (v3 — md 수정사항 반영)

변경사항 (v2 → v3):
  1. create_driver(): headless 관련 옵션 3줄 제거 → GUI Chrome으로 WebGL 정상 렌더링
  2. start_local_server(): 고정 포트(8765) → 포트 0 자동 할당 → 포트 충돌 방지
  3. SERVER_PORT: 전역 변수로 선언 → 함수 내부에서 global로 업데이트

사전 준비:
  1. https://developers.kakao.com 에서 JavaScript 앱키 발급
  2. 카카오 개발자 콘솔 → 내 애플리케이션 → 플랫폼 → Web → http://localhost 도메인 등록
  3. .env 파일에 KAKAO_JS_KEY=발급받은앱키 입력
  4. pip install selenium webdriver-manager pandas python-dotenv
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import pandas as pd
from dotenv import load_dotenv
import threading
import http.server
import socketserver

load_dotenv()
KAKAO_JS_KEY = os.getenv("KAKAO_JS_KEY")
SAVE_DIR     = "./roadview_images/2-3"
DATA_PATH    = "./schoolzone_location.csv"
MAX_ROWS     = 10        # None=전체, 숫자=테스트용
WAIT_SECONDS = 5         # 로드뷰 렌더링 대기 (초)
WINDOW_W     = 1280
WINDOW_H     = 720

os.makedirs(SAVE_DIR, exist_ok=True)

# ============================================================
# [수정 2] SERVER_PORT를 전역 변수로 선언
# → start_local_server() 내부에서 global로 업데이트할 수 있게 함
# ============================================================
SERVER_PORT = 0   # 실제 포트는 start_local_server() 실행 후 결정됨
_httpd = None


# ============================================================
# 0) 로컬 HTTP 서버 (백그라운드)
# ============================================================
def start_local_server():
    """
    [수정 2, 3] 포트 0으로 서버 생성 → OS가 빈 포트를 자동으로 할당.
    이전 서버가 같은 포트를 점유 중이어도 새 포트로 뜨므로
    WinError 10048(포트 충돌), WinError 10013(권한 거부) 모두 해결.
    """
    global _httpd, SERVER_PORT  # 전역 변수 수정 선언

    # 이미 서버가 실행 중이면 먼저 종료 시도
    if _httpd is not None:
        try:
            _httpd.shutdown()
            _httpd.server_close()
        except Exception:
            pass

    handler = http.server.SimpleHTTPRequestHandler
    socketserver.TCPServer.allow_reuse_address = True

    # ★ 핵심 변경: 고정 포트(8765) → 0 (OS 자동 할당)
    _httpd = socketserver.TCPServer(("", 0), handler)

    # OS가 실제로 할당한 포트 번호를 전역 변수에 저장
    SERVER_PORT = _httpd.server_address[1]

    thread = threading.Thread(target=_httpd.serve_forever, daemon=True)
    thread.start()
    print(f"🌐 로컬 서버 시작: http://localhost:{SERVER_PORT}")
    return _httpd


# ============================================================
# 1) 로드뷰 전용 HTML 생성
# ============================================================
def create_roadview_html(lat, lng, heading=0):
    """
    카카오 JS SDK로 로드뷰만 전체화면 렌더링하는 HTML 문자열 반환.
    - document.title로 로딩 완료/실패 신호 전달
      · LOADING        → 초기 상태
      · ROADVIEW_READY → 렌더링 완료 (캡처 가능)
      · NO_ROADVIEW    → 해당 좌표에 로드뷰 없음
    """
    return f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>LOADING</title>
    <style>
        * {{ margin: 0; padding: 0; }}
        html, body, #roadview {{ width: 100%; height: 100%; overflow: hidden; }}
    </style>
</head>
<body>
    <div id="roadview"></div>
    <script src="https://dapi.kakao.com/v2/maps/sdk.js?appkey={KAKAO_JS_KEY}"></script>
    <script>
        var container = document.getElementById('roadview');
        var roadview  = new kakao.maps.Roadview(container);
        var client    = new kakao.maps.RoadviewClient();
        var position  = new kakao.maps.LatLng({lat}, {lng});

        client.getNearestPanoId(position, 100, function(panoId) {{
            if (panoId === null) {{
                document.title = 'NO_ROADVIEW';
            }} else {{
                roadview.setPanoId(panoId, position);
                kakao.maps.event.addListener(roadview, 'init', function() {{
                    var vp  = roadview.getViewpoint();
                    vp.pan  = {heading};
                    vp.tilt = 0;
                    roadview.setViewpoint(vp);
                    setTimeout(function() {{
                        document.title = 'ROADVIEW_READY';
                    }}, 1500);
                }});
            }}
        }});

        // 10초 내 로드 안 되면 실패 처리
        setTimeout(function() {{
            if (document.title === 'LOADING') {{
                document.title = 'NO_ROADVIEW';
            }}
        }}, 10000);
    </script>
</body>
</html>"""


# ============================================================
# 2) 브라우저 생성
# ============================================================
def create_driver():
    """
    [수정 1] headless 관련 옵션 3줄 제거.
    
    제거된 옵션:
      --headless          → GUI 모드로 실행 (PC GPU 사용 가능)
      --disable-gpu       → GPU 활성화 (WebGL 하드웨어 렌더링)
      --use-gl=swiftshader → 소프트웨어 WebGL 에뮬레이터 비활성화

    headless 모드에서는 SwiftShader(소프트웨어 WebGL)를 사용하는데,
    카카오 로드뷰의 3D 파노라마는 SwiftShader에서 렌더링이 실패함.
    GUI Chrome으로 실행하면 PC의 실제 GPU를 사용해 정상 렌더링됨.
    """
    options = webdriver.ChromeOptions()
    # ★ --headless, --disable-gpu, --use-gl=swiftshader 3줄 제거
    options.add_argument(f"--window-size={WINDOW_W},{WINDOW_H}")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-software-rasterizer")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options,
    )
    return driver


# ============================================================
# 3) 로드뷰 캡처
# ============================================================
def capture_roadview(driver, lat, lng, facility_name, heading=0):
    """
    로컬 HTML을 localhost로 로드하여 로드뷰 캡처.
    document.title로 로딩 완료/실패를 판별.
    """
    safe_name = facility_name.replace("/", "_").replace(" ", "_")
    heading_labels = {0: "북", 90: "동", 180: "남", 270: "서"}
    label = heading_labels.get(heading, str(heading))

    # HTML 생성 → 임시 파일로 저장
    html_content = create_roadview_html(lat, lng, heading)
    html_path = os.path.abspath("_roadview_temp.html")
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_content)

    try:
        # [수정 2] SERVER_PORT가 동적으로 할당되므로 전역 변수 참조
        driver.get(f"http://localhost:{SERVER_PORT}/_roadview_temp.html")

        # 로드뷰 렌더링 완료 대기 (title 변경 감지)
        max_wait = WAIT_SECONDS + 5
        elapsed  = 0
        while elapsed < max_wait:
            title = driver.title
            if title == "ROADVIEW_READY":
                break
            elif title == "NO_ROADVIEW":
                print(f"  ⚠️  [{facility_name}] 로드뷰 없음 → 스킵")
                return None
            time.sleep(0.5)
            elapsed += 0.5

        if driver.title != "ROADVIEW_READY":
            print(f"  ⚠️  [{facility_name}] 로딩 타임아웃 → 스킵")
            return None

        # 추가 대기 (렌더링 안정화)
        time.sleep(1)

        # 스크린샷 저장
        path = f"{SAVE_DIR}/{safe_name}_{label}.png"
        driver.save_screenshot(path)

        # 유효성 검증 (10KB 미만이면 빈 이미지로 판단)
        file_size = os.path.getsize(path)
        if file_size < 10_000:
            os.remove(path)
            print(f"  ⚠️  [{facility_name}] {label}방향 → 빈 이미지, 삭제")
            return None

        print(f"  ✅ [{facility_name}] {label}방향 저장 → {path} ({file_size/1024:.0f}KB)")
        return path

    except Exception as e:
        try:
            alert = driver.switch_to.alert
            alert.accept()
        except Exception:
            pass
        print(f"  ❌ [{facility_name}] 오류: {e}")
        return None


# ============================================================
# 4) 전체 수집
# ============================================================
def collect_all():
    df = pd.read_csv(DATA_PATH)
    if MAX_ROWS is not None:
        df = df.head(MAX_ROWS)

    total         = len(df)
    results       = []
    success_count = 0
    skip_count    = 0
    headings      = [0]   # 필요시 [0, 90, 180, 270]으로 변경

    print("🌐 로컬 서버 시작...")
    httpd = start_local_server()

    print("🚀 브라우저 시작 (GUI 모드)...")
    driver = create_driver()
    print(f"📋 총 {total}개 스쿨존 수집 시작\n")

    try:
        for idx, row in df.iterrows():
            name = row["시설물명"]
            lat  = row["위도"]
            lng  = row["경도"]
            print(f"[{idx + 1}/{total}] {name} 처리 중...")

            saved_paths = []
            for h in headings:
                path = capture_roadview(driver, lat, lng, name, heading=h)
                if path:
                    saved_paths.append(path)

            if saved_paths:
                success_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "저장이미지수": len(saved_paths),
                    "파일경로": "; ".join(saved_paths),
                    "상태": "성공",
                })
            else:
                skip_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "저장이미지수": 0, "파일경로": "", "상태": "로드뷰없음",
                })
    finally:
        driver.quit()
        if os.path.exists("_roadview_temp.html"):
            os.remove("_roadview_temp.html")
        print("\n🛑 브라우저 종료")

    result_df   = pd.DataFrame(results)
    result_path = "./collection_result.csv"
    result_df.to_csv(result_path, index=False, encoding="utf-8-sig")

    print(f"\n{'='*50}")
    print(f"✅ 수집 완료!")
    print(f"   성공: {success_count}개 / 스킵: {skip_count}개 / 전체: {total}개")
    print(f"   결과: {result_path}")
    print(f"   이미지: {SAVE_DIR}")
    print(f"{'='*50}")
    return result_df


if __name__ == "__main__":
    result = collect_all()

🌐 로컬 서버 시작...
🌐 로컬 서버 시작: http://localhost:56927
🚀 브라우저 시작 (GUI 모드)...
📋 총 10개 스쿨존 수집 시작

[1/10] 중원초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 18:45:47] "GET /_roadview_temp.html HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2026 18:45:47] code 404, message File not found
127.0.0.1 - - [28/Feb/2026 18:45:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Feb/2026 18:45:50] code 404, message File not found
127.0.0.1 - - [28/Feb/2026 18:45:50] "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 -


  ⚠️  [중원초등학교] 로딩 타임아웃 → 스킵
[2/10] 하원초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 18:45:57] "GET /_roadview_temp.html HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2026 18:45:58] code 404, message File not found
127.0.0.1 - - [28/Feb/2026 18:45:58] "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 -


  ⚠️  [하원초등학교] 로딩 타임아웃 → 스킵
[3/10] 중부초등학교 처리 중...


...

  ⚠️  [돌마초등학교] 로딩 타임아웃 → 스킵
[9/10] 미금초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 18:47:09] "GET /_roadview_temp.html HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2026 18:47:09] code 404, message File not found
127.0.0.1 - - [28/Feb/2026 18:47:09] "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 -


  ⚠️  [미금초등학교] 로딩 타임아웃 → 스킵
[10/10] 백현초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 18:47:19] "GET /_roadview_temp.html HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2026 18:47:19] code 404, message File not found
127.0.0.1 - - [28/Feb/2026 18:47:19] "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 -


  ⚠️  [백현초등학교] 로딩 타임아웃 → 스킵

🛑 브라우저 종료

✅ 수집 완료!
   성공: 0개 / 스킵: 10개 / 전체: 10개
   결과: ./collection_result.csv
   이미지: ./roadview_images/2-3


In [13]:
import os
print("현재 작업 디렉토리:", os.getcwd())
print("HTML 파일 존재 여부:", os.path.exists("_roadview_temp.html"))

현재 작업 디렉토리: /Users/kyoungmin/ms_ai_school/project1/Road- view
HTML 파일 존재 여부: False


In [3]:
import os

# HTML 파일을 명시적인 절대경로로 저장해보기
html_path = "/Users/kyoungmin/ms_ai_school/project1/Road-view/_roadview_temp.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write("<html><body>test</body></html>")

print("파일 존재:", os.path.exists(html_path))

파일 존재: True


### 3-2) ✅ JS SDK + 일반 크롬
* 차이점
    - 로컬 PC GPU 사용 
    - 카카오맵 UI 없는 로드뷰 이미지 수집 가능
    - 속도 개선 (약 20s)            
=> 37s


In [ ]:
"""
카카오맵 로드뷰 Selenium 수집기 (v4 — 포트 고정)

v3 → v4 변경사항:
  - SERVER_PORT를 8080으로 고정
  - 카카오 콘솔에 http://localhost:8080 등록 필요
  - 포트 0 자동할당 제거 (카카오가 포트번호 포함해서 도메인 검증하기 때문)

사전 준비:
  1. 카카오 개발자 콘솔 → 웹 플랫폼 사이트 도메인에 http://localhost:8080 추가
  2. 카카오 개발자 콘솔 → JavaScript 키 → JS SDK 도메인에 http://localhost:8080 추가
  3. .env 파일에 KAKAO_JS_KEY="발급받은앱키" 입력
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import pandas as pd
from dotenv import load_dotenv
import threading
import http.server
import socketserver

load_dotenv()
KAKAO_JS_KEY = os.getenv("KAKAO_JS_KEY")
SAVE_DIR     = "./roadview_images/image_3-2"
DATA_PATH    = "./schoolzone_location.csv"
MAX_ROWS     = 10
WAIT_SECONDS = 5
WINDOW_W     = 1280
WINDOW_H     = 720

# 포트 고정 — 카카오 콘솔에 http://localhost:8080 등록 필요
SERVER_PORT  = 8080

os.makedirs(SAVE_DIR, exist_ok=True)
_httpd = None


def start_local_server():
    global _httpd
    if _httpd is not None:
        try:
            _httpd.shutdown()
            _httpd.server_close()
        except Exception:
            pass
    handler = http.server.SimpleHTTPRequestHandler
    socketserver.TCPServer.allow_reuse_address = True
    _httpd = socketserver.TCPServer(("", SERVER_PORT), handler)
    thread = threading.Thread(target=_httpd.serve_forever, daemon=True)
    thread.start()
    print(f"로컬 서버 시작: http://localhost:{SERVER_PORT}")
    return _httpd


def create_roadview_html(lat, lng, heading=0):
    return f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>LOADING</title>
    <style>
        * {{ margin: 0; padding: 0; }}
        html, body, #roadview {{ width: 100%; height: 100%; overflow: hidden; }}
    </style>
</head>
<body>
    <div id="roadview"></div>
    <script src="https://dapi.kakao.com/v2/maps/sdk.js?appkey={KAKAO_JS_KEY}"></script>
    <script>
        var container = document.getElementById('roadview');
        var roadview  = new kakao.maps.Roadview(container);
        var client    = new kakao.maps.RoadviewClient();
        var position  = new kakao.maps.LatLng({lat}, {lng});
        client.getNearestPanoId(position, 100, function(panoId) {{
            if (panoId === null) {{
                document.title = 'NO_ROADVIEW';
            }} else {{
                roadview.setPanoId(panoId, position);
                kakao.maps.event.addListener(roadview, 'init', function() {{
                    var vp  = roadview.getViewpoint();
                    vp.pan  = {heading};
                    vp.tilt = 0;
                    roadview.setViewpoint(vp);
                    setTimeout(function() {{
                        document.title = 'ROADVIEW_READY';
                    }}, 1500);
                }});
            }}
        }});
        setTimeout(function() {{
            if (document.title === 'LOADING') {{
                document.title = 'NO_ROADVIEW';
            }}
        }}, 10000);
    </script>
</body>
</html>"""


def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument(f"--window-size={WINDOW_W},{WINDOW_H}")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-software-rasterizer")
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options,
    )
    return driver


def capture_roadview(driver, lat, lng, facility_name, heading=0):
    safe_name = facility_name.replace("/", "_").replace(" ", "_")
    heading_labels = {0: "북", 90: "동", 180: "남", 270: "서"}
    label = heading_labels.get(heading, str(heading))

    html_content = create_roadview_html(lat, lng, heading)
    html_path = os.path.abspath("_roadview_temp.html")
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_content)

    try:
        driver.get(f"http://localhost:{SERVER_PORT}/_roadview_temp.html")
        max_wait = WAIT_SECONDS + 5
        elapsed  = 0
        while elapsed < max_wait:
            title = driver.title
            if title == "ROADVIEW_READY":
                break
            elif title == "NO_ROADVIEW":
                print(f"  [{facility_name}] 로드뷰 없음 -> 스킵")
                return None
            time.sleep(0.5)
            elapsed += 0.5

        if driver.title != "ROADVIEW_READY":
            print(f"  [{facility_name}] 로딩 타임아웃 -> 스킵")
            return None

        time.sleep(1)
        path = f"{SAVE_DIR}/{safe_name}_{label}.png"
        driver.save_screenshot(path)

        file_size = os.path.getsize(path)
        if file_size < 10_000:
            os.remove(path)
            print(f"  [{facility_name}] {label}방향 -> 빈 이미지, 삭제")
            return None

        print(f"  [{facility_name}] {label}방향 저장 -> {path} ({file_size/1024:.0f}KB)")
        return path

    except Exception as e:
        try:
            alert = driver.switch_to.alert
            alert.accept()
        except Exception:
            pass
        print(f"  [{facility_name}] 오류: {e}")
        return None


def collect_all():
    df = pd.read_csv(DATA_PATH)
    if MAX_ROWS is not None:
        df = df.head(MAX_ROWS)

    total         = len(df)
    results       = []
    success_count = 0
    skip_count    = 0
    headings      = [0]

    print("로컬 서버 시작...")
    start_local_server()

    print("브라우저 시작 (GUI 모드)...")
    driver = create_driver()
    print(f"총 {total}개 스쿨존 수집 시작\n")

    try:
        for idx, row in df.iterrows():
            name = row["시설물명"]
            lat  = row["위도"]
            lng  = row["경도"]
            print(f"[{idx + 1}/{total}] {name} 처리 중...")

            saved_paths = []
            for h in headings:
                path = capture_roadview(driver, lat, lng, name, heading=h)
                if path:
                    saved_paths.append(path)

            if saved_paths:
                success_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "저장이미지수": len(saved_paths),
                    "파일경로": "; ".join(saved_paths),
                    "상태": "성공",
                })
            else:
                skip_count += 1
                results.append({
                    "시설물명": name, "위도": lat, "경도": lng,
                    "저장이미지수": 0, "파일경로": "", "상태": "로드뷰없음",
                })
    finally:
        driver.quit()
        if os.path.exists("_roadview_temp.html"):
            os.remove("_roadview_temp.html")
        print("\n브라우저 종료")

    result_df   = pd.DataFrame(results)
    result_path = "./collection_result.csv"
    result_df.to_csv(result_path, index=False, encoding="utf-8-sig")

    print(f"\n{'='*50}")
    print(f"수집 완료!")
    print(f"   성공: {success_count}개 / 스킵: {skip_count}개 / 전체: {total}개")
    print(f"   결과: {result_path}")
    print(f"   이미지: {SAVE_DIR}")
    print(f"{'='*50}")
    return result_df


if __name__ == "__main__":
    result = collect_all()

로컬 서버 시작...
로컬 서버 시작: http://localhost:8080
브라우저 시작 (GUI 모드)...
총 10개 스쿨존 수집 시작

[1/10] 중원초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 20:14:35] "GET /_roadview_temp.html HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2026 20:14:35] code 404, message File not found
127.0.0.1 - - [28/Feb/2026 20:14:35] "GET /favicon.ico HTTP/1.1" 404 -


  [중원초등학교] 북방향 저장 -> ./roadview_images/2-3/중원초등학교_북.png (3440KB)
[2/10] 하원초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 20:14:38] "GET /_roadview_temp.html HTTP/1.1" 200 -


  [하원초등학교] 북방향 저장 -> ./roadview_images/2-3/하원초등학교_북.png (3066KB)
[3/10] 중부초등학교 처리 중...


...

  [돌마초등학교] 북방향 저장 -> ./roadview_images/2-3/돌마초등학교_북.png (3561KB)
[9/10] 미금초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 20:15:04] "GET /_roadview_temp.html HTTP/1.1" 200 -


  [미금초등학교] 북방향 저장 -> ./roadview_images/2-3/미금초등학교_북.png (3699KB)
[10/10] 백현초등학교 처리 중...


127.0.0.1 - - [28/Feb/2026 20:15:07] "GET /_roadview_temp.html HTTP/1.1" 200 -


  [백현초등학교] 북방향 저장 -> ./roadview_images/2-3/백현초등학교_북.png (4011KB)

브라우저 종료

수집 완료!
   성공: 10개 / 스킵: 0개 / 전체: 10개
   결과: ./collection_result.csv
   이미지: ./roadview_images/2-3
